In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt



In [2]:
# Check if CUDA is available and print the status
if torch.cuda.is_available():
    print("CUDA is available, using GPU")
    device = torch.device("cuda")
else:
    print("CUDA not available, using CPU")
    device = torch.device("cpu")

CUDA is available, using GPU


In [3]:
# Transform for data preprocessing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset and Dataloaders
train_dataset = datasets.ImageFolder(root='/kaggle/input/ddos-attack/attackImagesPaper/Train', transform=transform)
test_dataset = datasets.ImageFolder(root='/kaggle/input/ddos-attack/attackImagesPaper/Test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f'Number of classes: {len(train_dataset.classes)}')
print(f'Class to index mapping: {train_dataset.class_to_idx}')


Number of classes: 12
Class to index mapping: {'10_Syn.csvImgs': 0, '11_TFTP.csvImgs_12k': 1, '12_UDPLag_t.csvImgs': 2, '1_DrDoS_DNS.csvImgs': 3, '2_DrDoS_LDAP.csvImgs': 4, '3_DrDoS_MSSQL.csvImgs': 5, '4_DrDoS_NetBIOS.csvImgs': 6, '5_DrDoS_NTP.csvImgs': 7, '6_DrDoS_SNMP.csvImgs': 8, '7_DrDoS_SSDP.csvImgs': 9, '8_DrDoS_UDP.csvImgs': 10, '9_Normal_12k': 11}


In [4]:

model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 12)
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5, verbose=True)





/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 221MB/s]  
/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [5]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        if i % 100 == 99:
            print(f'Batch {i+1}, Loss: {running_loss/100:.3f}, Accuracy: {100.*correct/total:.2f}%')
            running_loss = 0.0
    
    epoch_accuracy = 100. * correct / total
    return epoch_accuracy, running_loss / len(dataloader)

In [6]:
def evaluate(dataloader, model, loss_fn):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    test_loss = running_loss / len(dataloader)
    test_accuracy = 100. * correct / total
    return test_loss, test_accuracy

In [7]:

num_epochs = 5
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    train_accuracy, train_loss = train_epoch(train_loader, model, loss_fn, optimizer)
    test_loss, test_accuracy = evaluate(test_loader, model, loss_fn)
    
    print(f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')
    print('-' * 40)
    
    # Learning rate scheduling
    scheduler.step(test_loss)

print('Training completed.')

Epoch 1/5
Batch 100, Loss: 1.551, Accuracy: 48.64%
Batch 200, Loss: 0.511, Accuracy: 64.83%
Batch 300, Loss: 0.533, Accuracy: 70.63%
Batch 400, Loss: 0.419, Accuracy: 74.23%
Batch 500, Loss: 0.436, Accuracy: 76.13%
Batch 600, Loss: 0.383, Accuracy: 77.95%
Batch 700, Loss: 0.317, Accuracy: 79.29%
Batch 800, Loss: 0.302, Accuracy: 80.38%
Batch 900, Loss: 0.306, Accuracy: 81.21%
Batch 1000, Loss: 0.313, Accuracy: 81.94%
Batch 1100, Loss: 0.329, Accuracy: 82.42%
Batch 1200, Loss: 0.346, Accuracy: 82.82%
Batch 1300, Loss: 0.281, Accuracy: 83.25%
Batch 1400, Loss: 0.292, Accuracy: 83.61%
Batch 1500, Loss: 0.269, Accuracy: 83.92%
Batch 1600, Loss: 0.271, Accuracy: 84.21%
Batch 1700, Loss: 0.288, Accuracy: 84.45%
Batch 1800, Loss: 0.336, Accuracy: 84.63%
Batch 1900, Loss: 0.257, Accuracy: 84.87%
Batch 2000, Loss: 0.295, Accuracy: 85.07%
Batch 2100, Loss: 0.253, Accuracy: 85.25%
Batch 2200, Loss: 0.278, Accuracy: 85.41%
Batch 2300, Loss: 0.238, Accuracy: 85.57%
Batch 2400, Loss: 0.246, Accuracy